In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
! tar -xvf '/Downloads/hi_fi_tts_v0.tar.gz' -C '/datasets/hi_fi_tts_v0/'
! cd '/datasets/hi_fi_tts_v0/'

tar: /Downloads/hi_fi_tts_v0.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
zsh:cd:1: no such file or directory: /datasets/hi_fi_tts_v0/


In [3]:
! pip install audiofile
! pip install pydub

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: voicefixer 0.1.2 has a non-standard dependency specifier streamlit>=1.12.0pyyaml. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of voicefixer or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade

In [5]:
from pathlib import Path
import os
from tqdm import tqdm
import audiofile
from pydub import AudioSegment
import json

In [10]:
import torchaudio
# coverts flac files to wavs and removes flac

def convert_path(audio_dir_path, target_suffix):
  if not audio_dir_path.is_dir():
    if audio_dir_path.suffix == target_suffix:
      return [audio_dir_path]
    return []
  out_list = []
  for path in audio_dir_path.iterdir():
    out_list += convert_path(path, target_suffix)
  return out_list

def convert_to_wavs(audio_path):
  flac_files = []
  for path in audio_path.iterdir():
    flac_files += convert_path(path, '.flac')
  for flac_path in tqdm(flac_files, 'flac -> wav'):
    wav_path = flac_path.parent / f'{flac_path.stem}.wav'
    flac_audio = AudioSegment.from_file(flac_path, "flac")
    flac_audio.export(wav_path, format="wav")
    os.remove(flac_path)
    
def change_samplerate(audio_path, target_sr):
  wav_files = []
  for path in audio_path.iterdir():
    wav_files += convert_path(path, '.wav')
  for wav_path in tqdm(wav_files, 'change samplerate'):
    wav_data, sr = torchaudio.load(wav_path)
    wav_data = torchaudio.functional.resample(wav_data, orig_freq=sr, new_freq=target_sr)
    torchaudio.save(wav_path, wav_data, target_sr)

In [4]:
import shutil
# remove other audio and files

def remove_other(dataset_path):
  for path in dataset_path.iterdir():
    if path.suffix == '.json' and 'other' in path.name:
      shutil.rmtree(path)

  audio_path = dataset_path / 'audio'
  for path in audio_path.iterdir():
    if 'other' in path.name:
      if path.is_dir():
        shutil.rmtree(path)
      else:
        os.remove(path)

In [10]:
import json

# creates meta.csv and fills it with data of the dataset
def create_meta_txt(dataset_path, out_txt_folder, speakers):
  out_txt_folder = Path(out_txt_folder)
  
  for speaker_id in speakers:
    speakers[speaker_id]['json_paths'] = []
  for path in dataset_path.iterdir():
    if path.suffix != '.json':
      continue
    assigned_speaker_id = None
    for speaker_id in speakers:
      if speaker_id in path.name:
        assigned_speaker_id = speaker_id
        break
    if assigned_speaker_id is None:
      print(f'Assigned speaker was not found: {path.name}')
      continue
    speakers[assigned_speaker_id]['json_paths'].append(path)
  
  for speaker_data in speakers.values():
    speaker_items = []
    for json_path in speaker_data['json_paths']:
      with open(json_path, 'r') as json_file:
        for row in json_file:
          json_data = json.loads(row)
          audio_path = Path(json_data['audio_filepath'])
          file_path = audio_path.parent / f'{audio_path.stem}.wav'
          global_file_path = dataset_path / file_path
          text = json_data['text_normalized']
          if global_file_path.exists():
            speaker_items.append((file_path, text, speaker_data['our_idx']))
          else:
            print(f'File not found for {file_path}')
    speaker_txt_path = out_txt_folder / f'{speaker_data["name"]}_audio_text.txt'
    with open(speaker_txt_path, 'w') as speaker_file:
      for wav, text, speaker_our_idx in speaker_items:
        speaker_file.write(f'{str(wav)}|{text}|{speaker_our_idx}\n')


In [4]:
from pathlib import Path

readers = {
    '9017': {'name': 'John_Van_Stan', 'our_idx': 1},
    '6097': {'name': 'Phil_Benson', 'our_idx': 2},
    '92': {'name': 'Cori_Samuel', 'our_idx': 3}
}

datasets_folder = Path('~/datasets').expanduser()
dataset_path = datasets_folder / 'hi_fi_tts_v0'

audio_path = dataset_path / 'audio'
transcript_path = dataset_path
out_txt_folder = Path('filelists')
# wavs_path = dataset_path / 'wavs'
# wavs_path.mkdir(exist_ok=True)

# remove_other(dataset_path)
# convert_to_wavs(audio_path)
# create_meta_txt(dataset_path, out_txt_folder, readers)
# change_samplerate(audio_path, 22050)


In [5]:
import numpy as np
from pathlib import Path

for _, speaker_data in readers.items():
    list_file_path = out_txt_folder / f'{speaker_data["name"]}_audio_text.txt'
    out_lines = []
    with open(list_file_path, 'r') as list_file:
        lines = list_file.readlines()
        for line in lines:
            line = line.strip()
            if len(line) == 0:
                continue
            parts = line.split('|')
            wav_path = Path('hi_fi_tts_v0') / parts[0]
            parts[0] = str(wav_path)
            pitch_path = Path('hi_fi_tts_v0') / 'pitch' / f'{wav_path.stem}.pt'
            
            assert (datasets_folder / wav_path).exists(), f'{datasets_folder / wav_path}'
            assert (datasets_folder / pitch_path).exists(), f'{datasets_folder / pitch_path}'
            
            parts.insert(1, str(pitch_path))
            out_lines.append('|'.join(parts) + '\n')
    list_file_path = out_txt_folder / f'{speaker_data["name"]}_audio_text.txt'
    with open(list_file_path, 'w') as list_file:
        list_file.writelines(out_lines)
    np.random.shuffle(out_lines)
    val_set = out_lines[:100]
    train_set = out_lines[100:]
    list_file_path = out_txt_folder / f'{speaker_data["name"]}_audio_pitch_text_train.txt'
    with open(list_file_path, 'w') as list_file:
        list_file.writelines(train_set)
    list_file_path = out_txt_folder / f'{speaker_data["name"]}_audio_pitch_text_val.txt'
    with open(list_file_path, 'w') as list_file:
        list_file.writelines(val_set)
    

AssertionError: /home/server2/datasets/hi_fi_tts_v0/hi_fi_tts_v0/audio/9017_clean/13885/dartagnan03part1_025_dumas_0071.wav

In [6]:
train_data = []
val_data = []

with open('filelists/ljs_audio_pitch_text_train_v3.txt', 'r') as list_file:
    lines = list_file.readlines()
    for line in lines:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split('|')
        wav_path = Path('LJSpeech-1.1') / parts[0]
        parts[0] = str(wav_path)
        pitch_path = Path('LJSpeech-1.1') / parts[1]
        parts[1] = str(pitch_path)
        
        assert (datasets_folder / wav_path).exists(), f'{datasets_folder / wav_path}'
        assert (datasets_folder / pitch_path).exists(), f'{datasets_folder / pitch_path}'
            
        train_data.append('|'.join(parts) + '\n')

with open('filelists/ljs_audio_pitch_text_val.txt', 'r') as list_file:
    lines = list_file.readlines()
    for line in lines:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split('|')
        wav_path = Path('LJSpeech-1.1') / parts[0]
        parts[0] = str(wav_path)
        pitch_path = Path('LJSpeech-1.1') / parts[1]
        parts[1] = str(pitch_path)
        
        assert (datasets_folder / wav_path).exists(), f'{datasets_folder / wav_path}'
        assert (datasets_folder / pitch_path).exists(), f'{datasets_folder / pitch_path}'
            
        val_data.append('|'.join(parts) + '\n')
        
train_list = []
val_list = []
for _, speaker_data in readers.items():
    train_list_file_path = out_txt_folder / f'{speaker_data["name"]}_audio_pitch_text_train.txt'
    val_list_file_path = out_txt_folder / f'{speaker_data["name"]}_audio_pitch_text_val.txt'
    train_list.append(train_list_file_path)
    val_list.append(val_list_file_path)
    
for train_list_file_path in train_list:
    with open(train_list_file_path, 'r') as train_list_file:
        lines = train_list_file.readlines()
        train_data += lines
with open(out_txt_folder / 'train.txt', 'w') as train_list_file:
    train_list_file.writelines(train_data)
    
for val_list_file_path in val_list:
    with open(val_list_file_path, 'r') as val_list_file:
        lines = val_list_file.readlines()
        val_data += lines
with open(out_txt_folder / 'val.txt', 'w') as val_list_file:
    val_list_file.writelines(val_data)

